In [ ]:
import pandas as pd
import chardet

# # 检测文件编码
# with open('./DataSet/Tokyo_20131_20234.csv', 'rb') as f:
#     result = chardet.detect(f.read())
#     encoding = result['encoding']
#     print(f"Detected encoding: {encoding}")

# 使用检测到的编码读取文件
df = pd.read_csv('../DataSet/Tokyo house price.csv', encoding="cp932")

In [ ]:
df.head()

In [ ]:
# 列名列表
columns = ['種類', '価格情報区分', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称','最寄駅：距離（分）', '取引価格（総額）', '坪単価', '面積（㎡）', '取引価格（㎡単価）', '土地の形状', '間口','延床面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類','前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）', '容積率（％）', '取引時期', '取引の事情等']

# 打印每一列的唯一值和唯一值的数量
for column in columns:
    unique_values = df[column].unique()
    unique_count = df[column].nunique()
    print(f"{column}: 有 {unique_count} 个唯一值。")
    print(f"这些唯一值是: {unique_values}")
    print()

In [ ]:
print(df.info)

In [ ]:
df.head()

In [ ]:
# 筛选出'取引時期'列中包含2018年至2023年的数据
df['取引時期'] = df['取引時期'].astype(str)
filtered_df = df[df['取引時期'].str.contains(r'202[0-3]|201[8-9]', regex=True)]
filtered_df = filtered_df .query('種類 == "宅地(土地と建物)"')
filtered_df = filtered_df .query('種類 == "宅地(土地と建物)"')
filtered_df = filtered_df .query('今後の利用目的 == "住宅"')

In [ ]:
selected_columns = ['最寄駅：距離（分）', '取引価格（総額）', '面積（㎡）','建築年', '建物の構造','地区名']
selected_df = df[selected_columns]

In [ ]:
selected_df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
# 确保列为字符串类型
for column in [ '建物の構造']:
    df[column] = selected_df[column].astype(str)

# 创建LabelEncoder对象
label_encoder = LabelEncoder()

# 对每个object列进行标签编码
for column in [ '建物の構造']:
    selected_df[column] = label_encoder.fit_transform(selected_df[column])

print(df)

In [ ]:
selected_df = selected_df.dropna()
#astype(int)

In [ ]:
# 提取建筑年份的年份部分并转换为整数
selected_df['建築年'] = selected_df['建築年'].str.extract('(\d+)')
selected_df = selected_df.dropna()
selected_df['建築年'] = selected_df['建築年'].astype(int)
# selected_df['建築年'] = selected_df['建築年'].astype(int)
# selected_df['建筑年限'] = 2024 - selected_df['建築年']

In [ ]:
selected_df.info()


In [ ]:
selected_df.to_csv('exported_data2.csv', index=False, encoding='utf-8')

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# 使用OneHotEncoder进行独热编码
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_features = one_hot_encoder.fit_transform(selected_df[['地区名']])

# 获取独热编码后的特征名
feature_names = one_hot_encoder.get_feature_names_out(['地区名'])

# 将编码后的特征转换为DataFrame
encoded_df = pd.DataFrame(encoded_features, columns=feature_names)

# 合并原始数据和独热编码后的数据
df_onehot = pd.concat([selected_df, encoded_df], axis=1)